In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ImportError: cannot import name '_raise_version_warning' from 'PIL' (/Users/mhihsan/miniconda3/envs/belajar/lib/python3.10/site-packages/PIL/__init__.py)

# Data Wrangling

## Gathering Data

In [ ]:
# Memuat tabel customers
customers_df = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/customers.csv")
customers_df.head()

In [ ]:
# Load tabel orders
orders_df = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/orders.csv")
orders_df.head()

In [ ]:
# Memuat tabel products
product_df = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/products.csv")
product_df.head()

In [ ]:
# Memuat tabel sales
sales_df = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/sales.csv")
sales_df.head()

## Assessing Data

### Menilai customers_df

In [ ]:
customers_df.info()

In [ ]:
customers_df.isna().sum()

In [ ]:
print("Jumlah duplikasi: ", customers_df.duplicated().sum())


In [ ]:
customers_df.describe()

### Menilai Orders_df

In [ ]:
orders_df.info()


In [ ]:
print("Jumlah duplikasi: ",orders_df.duplicated().sum())
orders_df.describe()

### Menilai Product_df

In [ ]:
product_df.info()


In [ ]:
print("Jumlah duplikasi: ", product_df.duplicated().sum())
 
product_df.describe()

### Menilai Sales_df

In [ ]:
sales_df.info()

In [ ]:
sales_df.isna().sum()


In [ ]:
print("Jumlah duplikasi: ", sales_df.duplicated().sum())
sales_df.describe()

## Cleaning Data

### Membersihkan data customers_df

In [ ]:
# Menghilangkan duplicate data
customers_df.drop_duplicates(inplace=True)

In [ ]:
print("Jumlah duplikasi: ", customers_df.duplicated().sum())


### Menangani Missing Value

In [ ]:
customers_df[customers_df.gender.isna()]


In [ ]:
customers_df.gender.value_counts()


In [ ]:
customers_df.fillna(value="Prefer not to say", inplace=True)


In [ ]:
customers_df.isna().sum()


In [ ]:
# Menangani inaccurate Value
customers_df[customers_df.age == customers_df.age.max()]

In [ ]:
# Ganti menjadi 70 tahun
customers_df.age.replace(customers_df.age.max(), 70, inplace=True)

In [ ]:
customers_df[customers_df.age == customers_df.age.max()]

In [ ]:
customers_df.describe()

In [ ]:
# Masih ada yang inaccurate
customers_df.age.replace(customers_df.age.max(), 50, inplace=True)

In [ ]:
customers_df.describe()

### Membersihkan Orders_df

In [ ]:
datetime_columns = ["order_date", "delivery_date"]
 
for column in datetime_columns:
  orders_df[column] = pd.to_datetime(orders_df[column])

In [ ]:
orders_df.info()


### Membersihkan Data product_df

In [ ]:
# Membersihkan 6 data yang duplicate
product_df.drop_duplicates(inplace=True)

In [ ]:
print("Jumlah duplikasi: ", product_df.duplicated().sum())

### Membersihkan Data sales_df

In [ ]:
sales_df[sales_df.total_price.isna()]

In [ ]:
sales_df["total_price"] = sales_df["price_per_unit"] * sales_df["quantity"]

In [ ]:
sales_df.isna().sum()


# Exploratory Data Analysis

Berdasarkan beberapa masalah tersebut, kita dapat mendefinisikan beberapa pertanyaan bisnis seperti berikut.

* Bagaimana performa penjualan dan revenue perusahaan dalam beberapa bulan terakhir?
* Produk apa yang paling banyak dan paling sedikit terjual?
* Bagaimana demografi pelanggan yang kita miliki?
* Kapan terakhir pelanggan melakukan transaksi?
* Seberapa sering seorang pelanggan melakukan pembelian dalam beberapa bulan terakhir?
* Berapa banyak uang yang dihabiskan pelanggan dalam beberapa bulan terakhir? 

## Explorasi Data customers_df

In [ ]:
customers_df.describe(include="all")


Berdasarkan rangkuman parameter statistik di atas, kita akan memperoleh informasi jumlah pelanggan sebanyak 1001 orang yang berumur di kisaran antara 20 hingga 80 tahun dengan rata-rata umur sebesar 49.87 tahun dengan standar deviasinya sebesar 17.64 tahun. Informasi ini tentunya dapat memberikan gambaran bahwa segmentasi pelanggan yang kita miliki cukup luas mulai dari remaja hingga lanjut usia.  

In [ ]:
customers_df.groupby(by="gender").agg({
    "customer_id": "nunique",
    "age": ["max", "min", "mean", "std"]
})

Berdasarkan pivot table di atas, dapat diketahui bahwa pelanggan yang kita miliki didominasi oleh jenis kelamin prefer not to say. Di lain sisi, distribusi umur mereka ternyata cukup mirip yaitu berkisar antara 20 hingga 80 tahun.

In [ ]:
customers_df.groupby(by="city").customer_id.nunique().sort_values(ascending=False)

In [ ]:
customers_df.groupby(by="state").customer_id.nunique().sort_values(ascending=False)

Berdasarkan hasil di atas, dapat dilihat bahwa persebaran pelanggan kita cukup merata pada setiap kota dan negara bagian. Pelanggan kita paling banyak berada di kota East Aidan, East Sophia, dan New Ava dengan jumlah masing-masing tiga pelanggan. Selain itu, pelanggan kita paling banyak berasal dari negara bagian South Australia.

## Explorasi data orders_df

Data kedua yang akan kita eksplor adalah orders_df. Ia mengandung berbagai informasi terkait sebuah order yang terdiri dari order_id, customer_id, order_date, dan delivery_date. Berdasarkan informasi tersebut, kita bisa membuat sebuah kolom baru untuk memuat waktu pengiriman tiap order. Untuk melakukannya, kita perlu menghitung selisih antar delivery_date dan order_date serta menyimpannya sebagai delivery_time. Selanjutnya, kita akan menggunakan method apply() untuk melakukan sebuah operasi terhadap setiap elemen dalam sebuah kolom DataFrame atau Series (bentuk satu dimensi dari DataFrame). Operasi yang akan kita lakukan ialah menghitung jumlah detik dari delivery_time menggunakan method total_seconds(). Nilai tersebut selanjutnya diubah ke dalam satuan hari (dibagi 86400) dan diambil bilangan bulatnya saja. Berikut merupakan contoh kode untuk melakukan semua proses tersebut.

In [ ]:
delivery_time = orders_df["delivery_date"] - orders_df["order_date"]
delivery_time = delivery_time.apply(lambda x: x.total_seconds())
orders_df["delivery_time"] = round(delivery_time/86400)

Setelah menjalankan kode di atas, Anda akan menemukan kolom baru pada DataFrame orders_df seperti berikut.

In [ ]:
orders_df.head()

In [ ]:
orders_df.describe(include="all")

Berdasarkan hasil tersebut, dapat diketahui bahwa rata-rata waktu pengiriman sebesar 14 hari dengan nilai maksimum sebesar 27 hari dan nilai minimum sebesar 1 hari. 

## Eksplorasi Data orders_df dan customers_df

pada data orders_df terdapat kolom yang berisi informasi terkait customer id pelanggan yang pernah melakukan order. Informasi ini bisa kita gunakan untuk mengidentifikasi pelanggan yang belum pernah melakukan order. Untuk melakukan hal ini, kita bisa membuat sebuah kolom baru bernama “status” pada data customers_df. Kolom tersebut memiliki nilai “Active” untuk pelanggan yang pernah melakukan order setidaknya sekali dan sebaliknya bernilai “Non Active” untuk pelanggan yang belum pernah melakukan order sama sekali. Berikut merupakan kode untuk melakukannya. 

In [ ]:
customer_id_in_orders_df =  orders_df.customer_id.tolist()
customers_df["status"] = customers_df["customer_id"].apply(lambda x: "Active" if x in customer_id_in_orders_df else "Non Active")
customers_df.sample(5)

In [ ]:
customers_df.groupby(by="status").customer_id.count()

Seperti yang terlihat pada pivot table di atas, terdapat cukup banyak pelanggan yang belum pernah melakukan transaksi sebelumnya. Ini tentunya merupakan kabar buruk bagi kita karena hampir 30% pelanggan kita belum pernah melakukan order sebelumnya.

In [ ]:
orders_customers_df = pd.merge(
    left=orders_df,
    right=customers_df,
    how="left",
    left_on="customer_id",
    right_on="customer_id"
)
orders_customers_df.head()

Pada kode di atas, kita melakukan proses merge dengan metode “left”. Tentunya Anda masih ingat bukan dengan metode tersebut. Yap, betul sekali metode ini memungkinkan kita untuk mengambil semua nilai dari tabel kiri serta nilai yang bersesuaian dengan tabel kanan. 

In [ ]:
orders_customers_df.groupby(by="city").order_id.nunique().sort_values(ascending=False).reset_index().head(10)


Nah, berdasarkan hasil tersebut, diketahui bahwa kota Jordanside dan New Ava merupakan dua kota yang memiliki jumlah order terbanyak.

In [ ]:
orders_customers_df.groupby(by="state").order_id.nunique().sort_values(ascending=False)

Kode tersebut akan menghasilkan pivot table seperti di bawah ini. Berdasarkan pivot table tersebut, diketahui bahwa South Australia merupakan state yang balik banyak melakukan order.

In [ ]:
orders_customers_df.groupby(by="gender").order_id.nunique().sort_values(ascending=False)

Berdasarkan hasil tersebut, dapat dilihat bahwa kebanyakan order dilakukan oleh pelanggan yang berjenis kelamin prefer not to say. Hal ini tentunya sejalan dengan jumlah pelanggan yang kebanyakan dari kelompok gender tersebut.

In [ ]:
orders_customers_df["age_group"] = orders_customers_df.age.apply(lambda x: "Youth" if x <= 24 else ("Seniors" if x > 64 else "Adults"))
orders_customers_df.groupby(by="age_group").order_id.nunique().sort_values(ascending=False)

Oke, sejauh ini kita telah menemukan banyak sekali informasi yang tentunya menarik untuk ditelusuri lebih dalam. Namun, sebelum itu mari kita mengeksplorasi dua data yang lain (product_df dan sales_df) terlebih dahulu. 

## Eksplorasi Data orders_df dan customers_df

Pada tahap ini, kita akan mengeksplorasi data product_df dan sales_df. Sebagai permulaan, kita dapat melihat rangkuman parameter statistik dari keduanya menggunakan method describe().

In [ ]:
product_df.describe(include='all')
sales_df.describe(include='all')

Berdasarkan hasil tersebut, dapat dilihat bahwa harga barang yang dijual berkisar antara 90 hingga 119 dolar. Selain itu, kita juga memperoleh informasi lain yang tidak kalah menarik yaitu pada setiap transaksi pelanggan paling banyak membeli tiga buah item dalam satu jenis produk dengan total harga sebesar 357 dolar. 

In [ ]:
product_df.sort_values(by="price", ascending=False)


Dapat dilihat bahwa produk termahal ialah item jaket bernama Parka dan yang termurah bernama Bomber.

Selanjutnya, kita bisa menggunakan pivot table untuk mencari informasi terkait produk berdasarkan tipe dan nama produknya. 

In [ ]:
product_df.groupby(by="product_type").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price":  ["min", "max"]
})

In [ ]:
product_df.groupby(by="product_name").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})

Pivot table di atas dapat memberikan kita gambaran terkait produk yang dijual oleh Dicoding Collection. Sebagai calon praktisi data yang Andal, tentunya Anda penasaran dengan produk yang paling laris. Nah, untuk menjawab pertanyaan ini, kita perlu menyatukan (merge) tabel product_df dan sales_df dengan kode berikut.

In [ ]:
sales_product_df = pd.merge(
    left=sales_df,
    right=product_df,
    how="left",
    left_on="product_id",
    right_on="product_id"
)
sales_product_df.head()

Pada dasarnya kode di atas akan melakukan proses merge terhadap data product_df dan sales_df. Proses merge tersebut dilakukan menggunakan metode “left”.

Jika diperhatikan kembali, hasil dari proses merge di atas memiliki perbedaan antara nilai price_per_unit dan price. Hal ini bisa terjadi karena adanya potongan harga, biaya operasional, dan biaya lainnya. 



In [ ]:
sales_product_df.groupby(by="product_type").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
})

Jika Anda perhatikan pivot table di atas, Trousers merupakan tipe produk yang paling laris. Akan tetapi, jika dilihat berdasarkan revenue yang diterima, Jacket merupakan tipe produk yang paling banyak menyumbang revenue perusahaan. 

In [ ]:
sales_product_df.groupby(by="product_name").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
}).sort_values(by="total_price", ascending=False)

Berdasarkan pivot table tersebut, diketahui bahwa produk Denim merupakan produk yang paling laris dan juga paling banyak menyumbang revenue terhadap perusahaan.  

## Eksplorasi Data all_df

Jika diperhatikan data penjualan ini cukup menarik untuk ditelusuri lebih dalam. Hal ini dilakukan guna melihat pola pembelian berdasarkan demografi pelanggan. Oleh karena itu, kita perlu membuat sebuah DataFrame baru bernama all_df untuk menampung semua informasi dari keempat tabel yang kita miliki. 

In [ ]:
all_df = pd.merge(
    left=sales_product_df,
    right=orders_customers_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
all_df.head()

Selanjutnya, mari kita coba lihat preferensi pembelian berdasarkan state pelanggan dan tipe produk menggunakan kode di bawah ini.

In [ ]:
all_df.groupby(by=["state", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

Pivot table di atas memberikan kita gambaran terkait tipe produk yang disukai pengguna berdasarkan lokasi negara bagiannya. Jika diperhatikan, tipe produk Jacket (garis merah) paling banyak terjual pada negara bagian Queensland, South Australia, dan New South Wales. Untuk tipe produk lain, Anda bisa melihatnya pada gambar pivot table di atas. 

Sekarang Anda bisa melakukan hal yang sama untuk mengetahui selera tipe produk pelanggan berdasarkan gender dan kelompok usia. 

In [ ]:
all_df.groupby(by=["gender", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

In [ ]:
all_df.groupby(by=["age_group", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

Nah, pivot table di atas dapat memberikan kita gambaran terkait selera tipe produk pelanggan berdasarkan gender dan kelompok usia. Semua informasi ini akan sangat membantu Anda dalam menjawab pertanyaan terkait preferensi pengguna berdasarkan demografinya.

Oke, sekarang kita selesai melakukan eksplorasi terhadap dataset yang kita miliki. Sejauh ini, masih banyak pertanyaan analisis atau bisnis yang belum terjawab. Namun, jangan khawatir. Pada materi selanjutnya, kita akan berkenalan dengan teknik visualisasi dan melakukan analisis data yang lebih advance guna menjawab pertanyaan bisnis yang belum terjawab tersebut.

# Visualisasi Data


### Bagaimana Performa Penjualan dan Revenue Perusahaan dalam Beberapa Bulan Terakhir
Kita perlu membuat sebuah DataFrame baru untuk menampung informasi terkait jumlah order dan total revenue yang diperoleh pada tiap bulannya. Oleh karena itu, kita perlu mengubah frekuensi dari data yang awalnya harian menjadi bulanan.



In [ ]:
monthly_orders_df = all_df.resample(rule='M', on='order_date').agg({
    "order_id": "nunique",
    "total_price": "sum"
})
monthly_orders_df.index = monthly_orders_df.index.strftime('%Y-%m')
monthly_orders_df = monthly_orders_df.reset_index()
monthly_orders_df.rename(columns={
    "order_id": "order_count",
    "total_price": "revenue"
}, inplace=True)
monthly_orders_df.head()

Oke, sekarang kita telah memiliki sebuah DataFrame yang memuat informasi yang dibutuhkan untuk menjawab pertanyaan pertama ini. Namun, untuk mempermudah orang lain dalam memahami informasi dari DataFrame tersebut, kita perlu membuat visualisasi datanya. Apakah Anda bisa menebak bentuk visualisasi data apa yang akan kita gunakan? Yap, betul sekali kita akan menggunakan bentuk line chart untuk memvisualkan informasi terkait jumlah order dan total revenue yang diperoleh tiap bulan. 

In [ ]:
monthly_orders_df = all_df.resample(rule='M', on='order_date').agg({
    "order_id": "nunique",
    "total_price": "sum"
})
monthly_orders_df.index = monthly_orders_df.index.strftime('%B') #mengubah format order date menjadi nama bulan
 
monthly_orders_df = monthly_orders_df.reset_index()
monthly_orders_df.rename(columns={
    "order_id": "order_count",
    "total_price": "revenue"
}, inplace=True)
 
plt.figure(figsize=(10, 5)) 
plt.plot(monthly_orders_df["order_date"], monthly_orders_df["order_count"], marker='o', linewidth=2, color="#72BCD4") 
plt.title("Number of Orders per Month (2021)", loc="center", fontsize=20) 
plt.xticks(fontsize=10) 
plt.yticks(fontsize=10) 
plt.show()

Berdasarkan visualisasi di atas, kita dapat melihat bahwa jumlah order terbanyak terjadi pada bulan Maret. Selain itu, kita juga dapat melihat adanya penurunan jumlah order yang cukup signifikan pada bulan Februari, April, Mei, dan Oktober. 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(
    monthly_orders_df["order_date"],
    monthly_orders_df["revenue"],
    marker='o', 
    linewidth=2,
    color="#72BCD4"
)
plt.title("Total Revenue per Month (2021)", loc="center", fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

### Produk Apa yang Paling Banyak dan Paling Sedikit Terjual?

In [ ]:
sum_order_items_df = all_df.groupby("product_name").quantity_x.sum().sort_values(ascending=False).reset_index()
sum_order_items_df.head(15)

Untuk mempermudah kita dalam menyampaikan informasi tersebut, kita harus membuat visualisasi data dalam bentuk bar chart. Selain itu, untuk mempermudah orang lain dalam mengidentifikasi produk dengan performa terbaik dan terburuk, kita perlu membuat dua buah visualisasi data dalam satu gambar visual. Untuk melakukan ini, gunakanlah function subplot(). Berikut contoh kode untuk membuatnya.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(24, 6))
 
colors = ["#72BCD4", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3"]
 
sns.barplot(x="quantity_x", y="product_name", data=sum_order_items_df.head(5), palette=colors, ax=ax[0], hue="product_name", legend=False)
ax[0].set_ylabel(None)
ax[0].set_xlabel(None)
ax[0].set_title("Best Performing Product", loc="center", fontsize=15)
ax[0].tick_params(axis ='y', labelsize=12)
 
sns.barplot(x="quantity_x", y="product_name", data=sum_order_items_df.sort_values(by="quantity_x", ascending=True).head(5), palette=colors, ax=ax[1], hue="product_name", legend=False)
ax[1].set_ylabel(None)
ax[1].set_xlabel(None)
ax[1].invert_xaxis()
ax[1].yaxis.set_label_position("right")
ax[1].yaxis.tick_right()
ax[1].set_title("Worst Performing Product", loc="center", fontsize=15)
ax[1].tick_params(axis='y', labelsize=12)
 
plt.suptitle("Best and Worst Performing Product by Number of Sales", fontsize=20)
plt.show()

### Bagaimana Demografi Pelanggan yang Kita Miliki?

In [ ]:
### Berdasarkan Gender
bygender_df = all_df.groupby(by="gender").customer_id.nunique().reset_index()
bygender_df.rename(columns={
    "customer_id": "customer_count"
}, inplace=True)
 
plt.figure(figsize=(10, 5))
 
sns.barplot(
    y="customer_count", 
    x="gender",
    data=bygender_df.sort_values(by="customer_count", ascending=False),
    palette=colors, hue="gender", legend=False
)
plt.title("Number of Customer by Gender", loc="center", fontsize=15)
plt.ylabel(None)
plt.xlabel(None)
plt.tick_params(axis='x', labelsize=12)
plt.show()

Berdasarkan gambar di atas, diketahui bahwa kebanyakan pelanggan tidak bersedia untuk memberitahukan informasi terkait gender-nya.

In [ ]:
### Berdasarkan age
byage_df = all_df.groupby(by="age_group").customer_id.nunique().reset_index()
byage_df.rename(columns={
    "customer_id": "customer_count"
}, inplace=True)
byage_df
byage_df['age_group'] = pd.Categorical(byage_df['age_group'], ["Youth", "Adults", "Seniors"])
plt.figure(figsize=(10, 5))
colors_ = ["#D3D3D3", "#72BCD4", "#D3D3D3", "#D3D3D3", "#D3D3D3"]
 
sns.barplot(
    y="customer_count", 
    x="age_group",
    data=byage_df.sort_values(by="age_group", ascending=False),
    palette=colors_
)
plt.title("Number of Customer by Age", loc="center", fontsize=15)
plt.ylabel(None)
plt.xlabel(None)
plt.tick_params(axis='x', labelsize=12)
plt.show()

In [ ]:
### Berdasarkan States
bystate_df = all_df.groupby(by="state").customer_id.nunique().reset_index()
bystate_df.rename(columns={
    "customer_id": "customer_count"
}, inplace=True)
bystate_df
plt.figure(figsize=(10, 5))
colors_ = ["#72BCD4", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3"]
sns.barplot(
    x="customer_count", 
    y="state",
    data=bystate_df.sort_values(by="customer_count", ascending=False),
    palette=colors_
)
plt.title("Number of Customer by States", loc="center", fontsize=15)
plt.ylabel(None)
plt.xlabel(None)
plt.tick_params(axis='y', labelsize=12)
plt.show()

### RFM Analysis

Untuk menjawab tiga pertanyaan analisis terakhir, kita bisa menggunakan teknik analisis lanjutan yang bernama RFM analysis. Sederhananya, RFM analysis merupakan salah satu metode yang umum digunakan untuk melakukan segmentasi pelanggan (mengelompokkan pelanggan ke dalam beberapa kategori) berdasarkan tiga parameter, yaitu recency, frequency, dan monetary.

* Recency: parameter yang digunakan untuk melihat kapan terakhir seorang pelanggan melakukan transaksi.
* Frequency: parameter ini digunakan untuk mengidentifikasi seberapa sering seorang pelanggan melakukan transaksi.
* Monetary: parameter terakhir ini digunakan untuk mengidentifikasi seberapa besar revenue yang berasal dari pelanggan tersebut.


In [ ]:
rfm_df = all_df.groupby(by="customer_id", as_index=False).agg({
    "order_date": "max", # mengambil tanggal order terakhir
    "order_id": "nunique", # menghitung jumlah order
    "total_price": "sum" # menghitung jumlah revenue yang dihasilkan
})
rfm_df.columns = ["customer_id", "max_order_timestamp", "frequency", "monetary"]
 
# menghitung kapan terakhir pelanggan melakukan transaksi (hari)
rfm_df["max_order_timestamp"] = rfm_df["max_order_timestamp"].dt.date
recent_date = orders_df["order_date"].dt.date.max()
rfm_df["recency"] = rfm_df["max_order_timestamp"].apply(lambda x: (recent_date - x).days)
 
rfm_df.drop("max_order_timestamp", axis=1, inplace=True)
rfm_df.head()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 6))
 
colors = ["#72BCD4", "#72BCD4", "#72BCD4", "#72BCD4", "#72BCD4"]
 
sns.barplot(y="recency", x="customer_id", data=rfm_df.sort_values(by="recency", ascending=True).head(5), palette=colors, ax=ax[0])
ax[0].set_ylabel(None)
ax[0].set_xlabel(None)
ax[0].set_title("By Recency (days)", loc="center", fontsize=18)
ax[0].tick_params(axis ='x', labelsize=15)
 
sns.barplot(y="frequency", x="customer_id", data=rfm_df.sort_values(by="frequency", ascending=False).head(5), palette=colors, ax=ax[1])
ax[1].set_ylabel(None)
ax[1].set_xlabel(None)
ax[1].set_title("By Frequency", loc="center", fontsize=18)
ax[1].tick_params(axis='x', labelsize=15)
 
sns.barplot(y="monetary", x="customer_id", data=rfm_df.sort_values(by="monetary", ascending=False).head(5), palette=colors, ax=ax[2])
ax[2].set_ylabel(None)
ax[2].set_xlabel(None)
ax[2].set_title("By Monetary", loc="center", fontsize=18)
ax[2].tick_params(axis='x', labelsize=15)
 
plt.suptitle("Best Customer Based on RFM Parameters (customer_id)", fontsize=20)
plt.show()

### Save All Data

In [ ]:
all_df.to_csv('all_data.csv', index=False)